 ## Create the data

In [1]:
# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis.enums import TimestampFormat
from mimesis import Datetime

dt = Datetime()
f = Field(locale=Locale.EN_GB, seed=42)


# Add table definitions

table_definition = {
    "person":{
        "amount":1000
    },
    "product":{
        "amount":1000
    },
    "order":{
        "amount":10000
    },
    "artist":{
        "amount":500
    },
    "review":{
        "amount":2000
    }
}

# Create table data

## person

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "id":"person:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person']['amount']
)
person_data = person_schema.create()

person_id_count = table_definition['person']['amount']-1

print("person data created")

## artist

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "id":"artist:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }


artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist']['amount']
)

artist_data = artist_schema.create()

artist_id_count = table_definition['artist']['amount']-1

print("artist data created")

## product

def product_generator() -> dict:
    created_at = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    quantity = f('integer_number', start=0, end=20)
    return {
        "id":"product:"+f"⟨{f('uuid')}⟩",
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=f('integer_number', start=8, end=25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":f('integer_number', start=0, end=20), 
        "image_url":f('stock_image_url'),
        "artist":artist_data[f('integer_number', start=0, end=artist_id_count)]['id'],
        "creation_history": {
            "created_at":created_at,
            "quantity":quantity
        }
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product']['amount']
)

product_data = product_schema.create()

product_id_count = table_definition['product']['amount']-1

print("product data created")

## order

def order_generator() -> dict:
    person_number = f('integer_number', start=0, end=person_id_count)
    product_number = f('integer_number', start=0, end=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    return {
        "id":f"order:"+f"⟨{f('uuid')}⟩",
        "in":person_data[person_number]['id'],
        "out":product_data[product_number]['id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity":f('integer_number', start=1, end=3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order']['amount']
)

order_data = order_schema.create()

order_id_count = table_definition['order']['amount']-1

print("order data created")

## review

def review_generator() -> dict:
    return {
        "id":"review:"+f"⟨{f('uuid')}⟩",
        "person":person_data[f('integer_number', start=0, end=person_id_count)]['id'],
        "product":product_data[f('integer_number', start=0, end=product_id_count)]['id'],
        "artist":artist_data[f('integer_number', start=0, end=artist_id_count)]['id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=f('integer_number', start=8, end=50)))
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review']['amount']
)

review_data = review_schema.create()

review_id_count = table_definition['review']['amount']-1

print("review data created")

person data created
artist data created
product data created
order data created
review data created


 ## Load the data

In [2]:
from surrealdb import SurrealDB
from uuid import uuid4

db = SurrealDB("ws://localhost:8000/test/test")

db.signin({
    "username": "root",
    "password": "root",
})


def insert_relate_statement(table_data:list[dict]) -> str:
    """
    Inserting data through relate statement
    """

    table_record_id = -1
    for record in table_data:
        table_record_id += 1
        db.query(
    f"RELATE {table_data[table_record_id]['in']} -> {table_data[table_record_id]['id']} -> {table_data[table_record_id]['out']} CONTENT {record};"
            )

In [3]:
%%time
db.query(f"INSERT INTO person {person_data}")

CPU times: user 3.28 s, sys: 83.6 ms, total: 3.36 s
Wall time: 3.43 s


[{'address': {'address_line_1': '1310 Blaney',
   'address_line_2': 'apt. 10',
   'city': 'Ballymena',
   'coordinates': [-84.279118, -146.269714],
   'country': 'England',
   'post_code': 'TG1C 7OP'},
  'email': 'holds1871@live.com',
  'first_name': 'Anthony',
  'id': 'person:⟨8d4b1d9e-bd8f-40cb-be78-fa2849abfd8f⟩',
  'last_name': 'Reilly',
  'name': 'Anthony Reilly',
  'phone': '01500 265774'},
 {'address': {'address_line_1': '477 Glenlevan',
   'city': 'Banbridge',
   'coordinates': [46.585326, -122.522646],
   'country': 'Scotland',
   'post_code': 'OS7K 4HW'},
  'email': 'dose1974@duck.com',
  'first_name': 'Laila',
  'id': 'person:⟨194651f5-fa66-44f2-83b2-02492f567444⟩',
  'last_name': 'Sears',
  'name': 'Laila Sears',
  'phone': '0112 713 3320'},
 {'address': {'address_line_1': '866 Derryview',
   'city': 'Luton',
   'coordinates': [55.283089, 82.703443],
   'country': 'Scotland',
   'post_code': 'EY3C 0WP'},
  'email': 'cargo1950@protonmail.com',
  'first_name': 'Vivien',
  'id

In [4]:
%%time
db.query(f"INSERT INTO product {product_data}")

[{'artist': 'artist:⟨fd69c9eb-687a-44d4-8806-77a500d0c480⟩',
  'category': 'pastel',
  'creation_history': {'created_at': '2023-09-19T00:35:25.767386',
   'quantity': 20},
  'currency': '£',
  'description': 'dr curve recognition please temple becomes hiv alliance crm leading alabama',
  'id': 'product:⟨48eacf79-cfd7-42bb-9f55-4c515341641e⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'page aims',
  'price': 4840.08,
  'quantity': 8},
 {'artist': 'artist:⟨27ca2491-a9fb-4597-b8f9-adfef2d46c46⟩',
  'category': 'acrylic paint',
  'creation_history': {'created_at': '2023-10-02T08:17:20.228523',
   'quantity': 7},
  'currency': '£',
  'description': 'marshall crystal mystery sh achieve served principle fear choice vintage famous bestsellers',
  'id': 'product:⟨1d47fc07-e245-4a71-b3ca-88f59d22b3dd⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'continental past',
  'price': 19434.78,
  'quantity': 17},
 {'artist': 'artist:⟨17407463-859d-4115-a56

In [5]:
%%time
insert_relate_statement(order_data)

CPU times: user 14.7 s, sys: 745 ms, total: 15.4 s
Wall time: 27 s


In [6]:
%%time
db.query(f"INSERT INTO artist {artist_data}")

CPU times: user 1.64 s, sys: 42.2 ms, total: 1.69 s
Wall time: 1.74 s


[{'address': {'address_line_1': '245 Kitcheners',
   'city': 'Harlow',
   'coordinates': [4.3413, -97.786325],
   'country': 'Northern Ireland',
   'post_code': 'JL9X 8VU'},
  'company_name': 'National Grid (NG.)',
  'email': 'catholic2093@protonmail.com',
  'first_name': 'Leonia',
  'id': 'artist:⟨dd0b56bd-9051-4ec9-b1a0-6f44330c1537⟩',
  'last_name': 'Dillon',
  'name': 'Leonia Dillon',
  'phone': '0141 389 8590'},
 {'address': {'address_line_1': '415 Drumanee',
   'city': 'Barry',
   'coordinates': [67.07081, 169.357399],
   'country': 'Scotland',
   'post_code': 'KC3B 5NT'},
  'company_name': 'Restaurant Gp (RTN)',
  'email': 'answers1955@outlook.com',
  'first_name': 'Wilburn',
  'id': 'artist:⟨ea26e500-f2a3-4c97-8649-fcd6cf78c60b⟩',
  'last_name': 'Christensen',
  'name': 'Wilburn Christensen',
  'phone': '0800 285089'},
 {'address': {'address_line_1': '115 Fennel',
   'city': 'Knighton',
   'coordinates': [-58.062479, 172.575897],
   'country': 'Scotland',
   'post_code': 'GU1Y 

In [7]:
%%time
db.query(f"INSERT INTO review {review_data}")

CPU times: user 866 ms, sys: 23.8 ms, total: 890 ms
Wall time: 983 ms


[{'artist': 'artist:⟨6590f52f-3b3a-474d-ad75-cefde6591e5d⟩',
  'id': 'review:⟨dcb17f4c-4564-477a-9394-e8c4844fb471⟩',
  'person': 'person:⟨2afc87a2-e332-4d24-b66c-f8aa11313a7b⟩',
  'product': 'product:⟨bd124d23-033e-446a-903a-83ab9a4dcda3⟩',
  'rating': 2,
  'review_text': 'displaying even screens closing paint serves surf expertise collaboration sons wonder click cause instruments greatest edwards nipples assistance speak commons made devil listening risks abc survival mn confirmation photos announces jr physicians manufacturers default neck jamaica solo regions boost filing wisdom deep'},
 {'artist': 'artist:⟨235d8648-5fec-44c3-af01-babb29466025⟩',
  'id': 'review:⟨7c8e9555-1714-44d6-b28c-26bf3ead4ae0⟩',
  'person': 'person:⟨ce23039d-0192-43ee-9173-a609ee9dda49⟩',
  'product': 'product:⟨7fe0b82b-40d2-4f11-9fb9-26c336f49d11⟩',
  'rating': 5,
  'review_text': 'charles visits scanner toshiba appears courts points python respective ha nature sufficient nepal park advertisers wars flowers

 ## Run the queries

In [ ]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = db.query(f"SELECT VALUE id FROM person")
product_ids = db.query(f"SELECT VALUE id FROM product")
order_ids = db.query(f"SELECT VALUE id FROM order")
artist_ids = db.query(f"SELECT VALUE id FROM artist")
review_ids = db.query(f"SELECT VALUE id FROM review")

 ### Q1: lookup vs record links

In [ ]:
%%timeit
list(db.query(""" 
SELECT
    id,
    rating,
    review_text,
	artist.name,
	artist.email,
	artist.phone,
	person.name,
	person.email,
	person.phone,
	product.name,
	product.category,
	product.price
FROM review;
"""))

 ### Q2: lookup vs graph (and link)

In [ ]:
%%timeit
db.query("""
SELECT
	price,
	order_date,
	product_name,
	->product.category,
	->product.description,
	->product.image_url,
	<-person.name,
	<-person.email,
	<-person.phone,
	->product.artist.name,
	->product.artist.email,
	->product.artist.phone
FROM order;
""")

 ### Q2: lookup vs graph - using in/out instead of arrow


In [ ]:
%%timeit
db.query(""" 
SELECT
	price,
	order_date,
	product_name,
	out.category,
	out.description,
	out.image_url,
	in.person.name,
	in.person.email,
	in.person.phone,
	out.artist.name,
	out.artist.email,
	out.artist.phone
FROM order;
""")

 ### Q3: Name and email for all customers in England

In [ ]:
# TODO add index on country
%%timeit
db.query(""" 
SELECT name, email 
FROM person 
WHERE address.country = "England";	
""")

 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [ ]:
# TODO add compound index on country, order_data and order_status
%%timeit
db.query(""" 
SELECT count() FROM order
WHERE (order_status != "pending"
OR order_status = null)
AND time::month(<datetime>order_date) <=3
AND ->product<-create<-artist.address.country ?= "England"
GROUP ALL;
""")


 ### Q5: Delete a specific review

In [ ]:
%%time
db.query(f""" 
DELETE {review_ids[0]};
""")


 ### Q6: Delete reviews from a particular category

In [ ]:
%%time
db.query(""" 
DELETE review
WHERE product.category = "charcoal";
""")

 ### Q7: Update a customer address

In [ ]:
%%timeit
db.query(f"UPDATE {person_ids[randint(0, person_id_count)]}"+"""
SET address = {
	'address_line_1': '497 Ballycander',
	'address_line_2': None,
	'city': 'Bromyard',
	'country': 'Wales',
	'post_code': 'ZX8N 4VJ',
	'coordinates': [68.772592, -35.491877]
	}
RETURN NONE;
""")

 ### Q8: Update discounts for products

In [ ]:
%%time
db.query(""" 
UPDATE product
SET discount = 0.2
WHERE price < 1000
RETURN NONE;
""")


 ### Q9: Transaction - order from a new customer

In [ ]:
%%timeit
random_person_id = person_ids[randint(0, person_id_count)]
random_product_id = product_ids[randint(0, product_id_count)]

db.query(""" 
# Transaction - order from a new customer
BEGIN TRANSACTION;
-- insert into the person table
CREATE person CONTENT {
	"""+f"'id': {random_person_id},"+"""
	'first_name': 'Karyl',
	'last_name': 'Langley',
	'name': 'Karyl Langley',
	'company_name': None,
	'email': 'dee1961@gmail.com',
	'phone': '+44 47 3516 5895',
	'address': {
		'address_line_1': '510 Henalta',
		'address_line_2': None,
		'city': 'Lyme Regis',
		'country': 'Northern Ireland',
		'post_code': 'TO6Q 8CM',
		'coordinates': [-34.345071, 118.564172]
		}
	};

-- relate into the order table"""+
f"RELATE {random_person_id} -> order:uuid() -> {random_product_id}"+"""
CONTENT {
        "currency": "£",
        "discount": ->product.discount,
        "order_date": time::now(),
        "order_status": "pending",
        "payment_method": "PayPal",
        "price": ->product.price,
        "product_name": ->product.name,
        "quantity": 1,
        "shipping_address": <-person.address
	};

-- update the product table to reduce the quantity"""+
f"""
UPDATE {random_product_id} SET quantity -= 1 RETURN NONE;
COMMIT TRANSACTION;
""")


 ### Q10: "Transaction"* - New Artist creates their first product

In [ ]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = str(uuid4())
new_product_id = str(uuid4())

db.query(""" 
BEGIN TRANSACTION;
-- insert into the artist table
CREATE artist CONTENT {"""+
        f"'id': 'artist:⟨{new_artist_id}⟩',"+"""
        'first_name': 'Anderson',
        'last_name': 'West',
        'name': 'Anderson West',
        'company_name': 'Atkins(ws) (ATK)',
        'email': 'six1933@gmail.com',
        'phone': '056 5881 1126',
        'address': {
                'address_line_1': '639 Connaugh',
                'address_line_2': None,
                'city': 'Ripon',
                'country': 'Scotland',
                'post_code': 'CG3U 4TH',
                'coordinates': [4.273648, -112.907273]
                }
        };

-- insert into the product table
CREATE product CONTENT {"""+
        f"'id': 'product:⟨{new_product_id}⟩',"+"""
        'name': 'managed edt allocated pda',
        'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
        'category': 'watercolor',
        'price': 15735.96,
        'currency': '£',
        'discount': None,
        'quantity': 1,
        'image_url': 'https://source.unsplash.com/1920x1080?'
        };

-- relate into the create table"""+
f"RELATE artist:⟨{new_artist_id}⟩ -> create:uuid() -> product:⟨{new_product_id}⟩"+"""
CONTENT {
        created_at: time::now(),
        quantity: 1
};
COMMIT TRANSACTION;
""")